In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline


In [2]:
import numpy
import scipy
import matplotlib as mpl
import matplotlib.dates as mpd
import pylab as plt
import datetime
import os,sys
#
import subprocess
import requests
import tarfile
import shutil
#
import urllib
import contextlib
# import urllib.request as request
# from contextlib import closing
#
import re
#
import json
import netCDF4
#
import AM4py

### AM4 runtime scripting: Development, examples, and templates

This notebook contains development work, examples, and templates to script AM4 simulations in Python. The basic strategy is to write a Python script to manage preliminary setup tasks, ie setting up working directories, copying data, assessing the restart status, and then (possibly) submit a simple batch script to run the large MPI job.

Tasks delegated to the Python parent script include:

- Do we have a working directory? Does it contain an INPUT folder?
- If not, copy INPUT data from a designated source (which can take a long, long time).
- Configure the batch scritp and `.nml` configuration to:
    - Use the correct `layout` formats
    - Request the correct number of processors
    - Make any configuratioon changes, from a base template
    - Other stuff too...
- Manage restarts

The advantages of this approach are:

1. Python is generally considered a more versatile and easy to use scripting language than any shell language.
2. To that point, most of these projects still come with `csh` or `tcsh` scripts, which are often simply not supported by newer OSs, including RedHat 7.x (it might be nominally available, but 1) might not be supported by admins (ie, LMOD will not work), and 2) it can in many cases just plain not run correctly. Accordingly, it is often necessary to translate any standard or provided scripts to `bash` anyway. We hope that, with these tools as a starting place, it will be easier to skip straight to Python.
3. Shell scripting often does not directly support very simple logic, that can be very helpful when managing large, complex runs -- particularly with checkpointing. For example, floating point math is not directly supported in `bash`.
4. Some preliminary operations, for example acquiring or copying input data, can be extremely cumbersome (it's a lot of data!); this approach allows these steps to be accomplished by a single-process setup script before the large MPI job is submitted. This will help to maintain friendships with your colleagues and sysadmins.
5. Permits flexibility, more complex scripts, for example running in small checkpointed steps, to better utilize shared resources, and (possibly) saving the incremental output.



### AM4py Example

- OPTIONAL:
    - Define big-picture (ie, multiple runs) job
    - Evaluate `work_dir` data to determine next actions (ie, did the previous job run, are we finished, etc.)
- Instantiate an `AM4_batch_scripter` object
    - It will evaluate whether the input data exist; if not, it will go get it.
- Modify the NML configuration and export a working NML
- Write a batch file to run AM4
- Submit that script


In [6]:
print(os.environ['HOME'])

/Users/myoder96


In [7]:
#
input_data_path = os.path.join(os.environ['HOME'], 'Codes', 'AM4_data2', 'AM4_run')
work_dir = os.path.join(os.environ['HOME'], 'Codes', 'AM4_runtime', 'workdir2')
job_name = 'AM4_dev'
batch_job_name = os.path.join(work_dir, 'AM4_batch_example.bs')
#
#print('*** input_data_path: ', input_data_path)
#
ABS = AM4py.AM4_batch_scripter(input_data_path=imput_data_path, work_dir=work_dir,
                               job_name='AM4_dev', batch_out=batch_job_name )
#
print('*** ABS variables:')
for key,val in ABS.__dict__.items():
    print('{}: {}'.format(key,val))
#

zz = ABS.get_input_data(verbose=True)


my_configs = {'coupler_nml':{'days':10, 'months':0}, 'fv_core_nml':{'npx':193, 'npy':193, 'npz':50}}
my_nml = ABS.make_NML(nml_template='input_yoder_v101.nml', nml_configs=[my_configs],
                      nml_out=os.path.join(ABS.work_dir, 'input.nml') )
# NML_from_nml('input_yoder_v101.nml')

#my_nml = NML_from_nml('input_yoder_v101.nml')
print('** my_nml[fv_core_nml]:' )
print('** ', my_nml['fv_core_nml'])
#print(my_nml.keys())

print('** batch_out: ', ABS.batch_out)
#
ABS.write_batch_script()

*** input_data_path:  /Users/myoder96/Codes/AM4_data2/AM4_run


In [8]:
print('** ', ABS.batch_out)

**  am4_batch.sh


In [5]:
print(ABS.mpi_exec)

print('MPI_COMMAND={} {}{} {}{} ${{EXECUTABLE\}}'.format(ABS.mpi_exec['exec'],
                                                            ABS.mpi_exec['ntasks'], ABS.n_tasks,
                                                            ABS.mpi_exec['cpu_per_task'], ABS.n_threads))

{'exec': 'mpirun', 'ntasks': '--np ', 'cpu_per_task': '-d '}
MPI_COMMAND=mpirun --np 48 -d 1 ${EXECUTABLE\}


In [ ]:
# simplified syntax to capture both stdout and stderr (to separate outputs?). NOTE: this syntax has been
#. a moving target
#sp_output = subprocess.run('ls -lh'.split(chr(32)), check=True, capture_output=True)
#
# pipe stdout and stderr to the same output
sp_output = subprocess.run('ls -lh /Users/myoder96/Codes/AM4_data2/AM4_run/'.split(chr(32)), check=True,
                           stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
#
print('** sp_output: \n', sp_output)
# for rw in sp_output.stdout.decode().split('\n'):
#     print*('** ', rw)
print('** ** stdout: \n', sp_output.stdout.decode())
for rw in sp_output.stdout.decode().split('\n'):
    print('** ', rw)

In [ ]:
NML_test = NML_from_nml('input_yoder_v101.nml')
#

# for ky,vl in JJ.nml_dict.items():
#     print('** {}: {}'.format(ky,vl))


for ky in ['coupler_nml','coupler_nml', 'vegn_data_nml', 'simple_sulfate_nml' ]:
    print('*** {}:{}\n'.format(ky, NML_test[ky]))

In [ ]:
print(NML_test['aerosolrad_package_nml']['sulfate_indices'])

### Working NML example.

- Start with a standard template
- Compute layouts for an MPI configuration
- Modify layout variables (in internal JSON/dict)
- Export working input.nml

In [ ]:
        
n_tasks = 48
n_threads = 1
#
layouts_1 = get_layouts(n_tasks)
layouts_2 = get_layouts(n_tasks = n_tasks/6)
#
layout_io_1 = get_io_layouts(layouts_1[0])
layout_io_2 = get_io_layouts(layouts_2[0])
#
print('** Layouts_1: ', layouts_1)
print('** Layouts_2: ', layouts_2)
#
print('** Layouts_io_1: ', layout_io_1)
print('** Layouts_io_2: ', layout_io_2)
#
my_nml = NML_from_nml(input_nml='input_yoder_v101.nml')

In [ ]:
print('*** ', my_nml.keys())

In [ ]:
layout_1=layouts_1[0]
layout_2=layouts_2[0]
#
layout_io = layout_io_1[0]

#
# print out layouts, as they are imported:
for grp in ('fv_core_nml', 'land_model_nml','ocean_model_nml', 'ice_model_nml'):
    print('** {}::layout: {}'.format(grp, my_nml[grp]['layout']))
    print('** {}::io_layout: {}'.format(grp, my_nml[grp]['layout']))
    print('\n')

In [ ]:
print('** ', 48%6)
print('** ', 48%5)

In [ ]:
#print('** ', my_nml['fv_core_nml']['layout'])
#
for grp in ('fv_core_nml', 'land_model_nml'):
    my_nml.assign(grp, 'layout', ','.join([str(x) for x in layout_2]))
    my_nml.assign(grp, 'io_layout', ','.join([str(x) for x in layout_io]))
    print('** {}:: {}, {}'.format(grp, my_nml[grp]['layout'], my_nml[grp]['io_layout']))
#
for grp in ('ocean_model_nml', 'ice_model_nml'):
    my_nml.assign(grp, 'layout', ','.join([str(x) for x in layout_1]))
    my_nml.assign(grp, 'io_layout', ','.join([str(x) for x in layout_io]))
    print('** {}:: {}, {}'.format(grp, my_nml[grp]['layout'], my_nml[grp]['io_layout']))
#
for ky,vl in [('npx',193), ('npy', 193), ('npz', 50)]:
    my_nml.assign('fv_core_nml', ky, vl)
#
for ky,vl in [('co2_ceiling', 4800.0E-06), ('time_varying_co2', '.true.'), ('co2_base_value',348.0E-06),
            ('co2_floor', 100.0E-06), ('c02_data_source', 'namelist') ]:
    # NOTE: we want to allow new assignment:
    my_nml['radiative_gases_nml'][ky] = vl
print('*** radiative_gases: ', my_nml['radiative_gases_nml'])

In [ ]:
for ky,vl in my_nml['radiative_gases_nml'].items():
    print('** {}:: {}'.format(ky, vl))
print('\n\n')
#
for ky,vl in my_nml['fv_core_nml'].items():
    print('** {}:: {}'.format(ky, vl))

In [ ]:
my_nml.json_to_nml(nml_out='my_output.nml', json_in=my_nml)

In [ ]:
print(','.join([str(x) for x in [1,2,3]]))